In [13]:
from __future__ import print_function

import logging
import os

import SimpleITK as sitk
import six
import numpy as np

import radiomics
from radiomics import featureextractor, getFeatureClasses
from radiomics import glcm
from radiomics.featureextractor import RadiomicsFeatureExtractor
from tqdm import tqdm


In [14]:
radiomics.setVerbosity(logging.INFO)
radiomics.progressReporter = tqdm

In [15]:
kernel = 2
# size = 24

# imageName = sitk.GetImageFromArray(np.random.randn(size, size, size))
# # maskV = np.ones((size, size, size))
# # maskV[0][0][0] = 0
# maskName = sitk.GetImageFromArray(np.ones((size, size, size)))

In [16]:
maskPath='dataset/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_seg.nii.gz'
maskName = sitk.ReadImage(maskPath)
imagePath = 'dataset/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_flair.nii.gz'
imageName=sitk.ReadImage(imagePath)

In [17]:
def get_default_features():
    return None
    # return ['JointEntropy']
    # return ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade',
    #         'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy',
    #         'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2',
    #         'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares']

def get_default_settings():
    settings = {}

    # geometryTolerance = 1e-1
    # settings["geometryTolerance"] = geometryTolerance
    # sitk.ProcessObject.SetGlobalDefaultCoordinateTolerance(geometryTolerance)
    # sitk.ProcessObject.SetGlobalDefaultDirectionTolerance(geometryTolerance)

    # settings["normalize"] = True
    # settings["normalizeScale"] = 100
    # settings["removeOutliers"] = None
    
    # settings["resampledPixelSpacing"] = None # We have already resampled
    # # settings["resampledPixelSpacing"] = [1, 1, 1]
    # settings["interpolator"] = sitk.sitkBSpline

    # settings["binWidth"] = 25
    # settings["voxelArrayShift"] = 300
    # settings["enableCExtensions"] = True
    
    return settings

# imageType:
#   Original: {}

# featureClass:
#   glcm:
#     - JointEntropy

# setting:
#   binWidth: 25
#   force2D: true
#   label: 1

# voxelSetting:
#   kernelRadius: 2
#   maskedKernel: true
#   initValue: nan
#   voxelBatch: 10000
# def get_default_settings():
#     settings = {}

#     return settings

In [18]:
# glcm.RadiomicsGLCM
# rf_ext = RadiomicsFeatureExtractor(
#         voxelBased=True, maskedKernel=False, voxelBatch=512,
#         **get_default_settings())

# img_norm, mask_norm = rf_ext.loadImage(imageName, maskName, None, **rf_ext.settings)

# a_ext = glcm.RadiomicsGLCM(
#         img_norm, mask_norm,
#         voxelBased=True, padDistance=kernel,
#         kernelRadius=kernel, maskedKernel=False, voxelBatch=1024,
#         **get_default_settings())
a_ext = glcm.RadiomicsGLCM(
        imageName, maskName,
        voxelBased=True, maskedKernel=False, voxelBatch=1024,
        **get_default_settings())

if get_default_features() is not None:
        a_ext.disableAllFeatures()
        for n in get_default_features():
            a_ext.enableFeatureByName(n)

a = a_ext.execute()

batch: 100%|██████████| 12/12.0 [01:20<00:00,  6.74s/it]


In [19]:
a

{'JointEntropy': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x000001AE3E053D20> >}

In [20]:
featureMap = a['JointEntropy']
featureData = sitk.GetArrayFromImage(featureMap)

In [21]:
featureData.shape

(155, 240, 240)

In [22]:
featureData.shape

(155, 240, 240)

In [23]:
# featureData.
np.where(featureData == 4.53996538528106)

(array([61, 62, 62, 63, 72, 72, 73, 73, 75, 76, 78, 79, 79, 79, 84],
       dtype=int64),
 array([100,  73,  74,  84,  84,  85,  74,  75,  90,  87,  74,  74,  74,
         88,  92], dtype=int64),
 array([144, 137, 137, 132, 149, 152, 145, 145, 148, 131, 144, 141, 144,
        132, 139], dtype=int64))

In [24]:
featureData[61][100][144]

4.53996538528106

In [25]:
featureData.mean()

0.005851361887708959